<img src="https://dialektico.com/wp-content/uploads/2023/03/MiniLogoW4.png" alt="Dialéktico Logo" />

Este pequeño tutorial pertenece al curso de RAG y agentes con LangChain al que puedes acceder mediante la siguiente URL: https://www.youtube.com/playlist?list=PLlWTv9_GeWd32stuEMWpYOnxiVxnXaU6q

Sigue los videos del curso para recibir instrucciones y contexto sobre la ejecución de este Notebook.

**Nota**: este notebook está pensado para ser ejecutado en local, permitiendo una conexión de este tipo a una base de datos SQL.

<br>

# Creación de la base de datos

Primero se debe crear una base de datos local, para esto recomiendo utilizar DBeaver, una plataforma gratuita y fácil de usar para administración de base de datos. 

Aquí puedes descargar el programa: https://dbeaver.io/download/

Después, se debe crear la base de datos siguiendo estos pasos:

* Abre DBeaver.

* Da clic en "Nueva conexión", en el ícono de enchufe bajo el menu "Archivo".

* Elige SQLite (puedes buscarlo con el buscador).

* Da clic en Siguiente.

* En la pantalla "General", da clic en Path y elige una carpeta local donde guardar la base de datos.

* Escribe chat_history.db como nombre de la base de datos.

* Da clic en Finalizar.

# Instalación de librerías

Ahora debes crear un entorno virtual, para esto abre la cmd, navega hasta el directorio donde esté el código alojado, y ejecuta:

```python -m venv venv```

Después, actívalo con:

```venv\Scripts\activate```

Finalmente, instala las librerías:

```pip install langchain_deepseek==0.1.2 langchain-community==0.3.20```

# Se importan las librerías

In [4]:
import os
from pathlib import Path
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import SQLChatMessageHistory
from langchain_deepseek import ChatDeepSeek

Ahora realiza una conexión a la base de datos, pasando la ruta de la base de datos anteriormente creada.

In [ ]:
# Define la ruta usando Path y se convierte a formato URI
db_path = Path("C:/Users/Usuario/Documents/Data Science/Curso RAG/chat_history.db").resolve()
connection_string = f"sqlite:///{db_path.as_posix()}"

# Se realiza la conexión a la base de datos.
chat_message_history = SQLChatMessageHistory(
    session_id="1416", connection =connection_string
)

Almacena un par de mensajes.

In [ ]:
# Almacena dos mensajes en la base de datos

chat_message_history.add_user_message("Hola")
chat_message_history.add_ai_message("Hola")

In [5]:
chat_message_history.messages

[HumanMessage(content='Hola', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Hola', additional_kwargs={}, response_metadata={})]

## Se añade valor de API key del modelo a utilizar.

In [16]:
os.environ["DEEPSEEK_API_KEY"] = "sk-9b3b410681f446f1a102fa3c96920005"

## Se declara el modelo a utilizar

In [17]:
# Se define el modelo y añaden valores de parámetros.
model = ChatDeepSeek(
      model="deepseek-chat",
      temperature=0,
      max_tokens=100
      )

## Se crea la cadena a utilizar para la generación de mensajes.

In [18]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "Eres un asistente amable."),
        MessagesPlaceholder(variable_name="history"),
        ("user", "{question}"),
    ]
)

chain = prompt | model

# Se crea una cadena con memoria persistente

Se debe crear una instancia de RunnableWithMessageHistory, la cual permite al LLM responder conforme a los mensajes almacenados, segmentados por un session_id.

In [19]:
chain_with_history = RunnableWithMessageHistory(
    chain,
    lambda session_id: SQLChatMessageHistory(
        session_id=session_id, connection_string=connection_string
    ),
    input_messages_key="question",
    history_messages_key="history",
)

Se añade un diccionario con el ID que distinguirá el grupo de mensajes para la memoria conversacional.

In [24]:
config = {"configurable": {"session_id": "123"}}

# Se realizan las pruebas de memoria

In [25]:
chain_with_history.invoke({"question": "Hola, mi nombre es Dialéktico"}, config=config).content

'¡Hola, Dialéktico! Me alegra conocerte. ¿En qué puedo ayudarte hoy? 😊  \n\nSi buscas profundizar en algún tema, debatir ideas o simplemente conversar, estoy aquí para lo que necesites. ¿Qué te gustaría explorar?'

In [26]:
chain_with_history.invoke({"question": "Gracias, recuérdame cuál es mi nombre"}, config=config).content

'¡Claro! Tu nombre es **Dialéktico** (y vaya que es un nombre con mucha profundidad filosófica, por cierto 😉).  \n\n¿Hay algo más en lo que pueda ayudarte, Dialéktico? Estoy aquí para lo que necesites. ✨'

Más información en:
https://python.langchain.com/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html
https://python.langchain.com/docs/integrations/memory/sqlite/

<br>

In [ ]:
# Dialektico Machine learning practices © 2025 by Daniel Antonio García Escobar
# is licensed under CC BY-NC 4.0. To view a copy of this license,
# visit https://creativecommons.org/licenses/by-nc/4.0/

# Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International
# Public License